**Nombre:** Andrea Aguilera

**Trabajo Final de Materia Tópicos Especiales:** Sistema de Recuperación Aumentada por Generación (RAG) para la Primera Infancia

In [ ]:
#Instalar librerías
!pip install -U langchain langchain-groq langchain-community faiss-cpu python-dotenv sentence-transformers PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.7/345.7 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.5/127.5 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 437.6/437.6 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 99.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 78.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8

In [ ]:
pip install -U langchain-huggingface


In [ ]:
import os
from dotenv import load_dotenv
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_groq import ChatGroq

In [ ]:
# 1. Configuración de entorno

from google.colab import drive
import dotenv

# Montar Google Drive
drive.mount('/content/drive')

# Definir la ruta base del proyecto
BASE_DIR = "/content/drive/MyDrive/ColabNotebooks/NLP/Trabajo_Final"
pdf_dir = os.path.join(BASE_DIR, "corpus")

# Cargar variables de entorno desde el .env
dotenv.load_dotenv('/content/drive/MyDrive/ColabNotebooks/NLP/Trabajo_Final/.env', override=True)

# Obtener la API key
api_key = os.getenv('GROP_API_KEY')

# Verificar
if api_key:
    print("🔐 API Key cargada correctamente ✅")
    print(api_key[:10] + "..." + api_key[-4:])
else:
    print("❌ No se encontró la API Key. Revisá la ruta o el contenido del archivo.")


Mounted at /content/drive
🔐 API Key cargada correctamente ✅
gsk_TPCYpx...uvc1


In [ ]:
# 2. Cargar todos los PDFs
all_docs = []

for file in os.listdir(pdf_dir):
    if file.endswith(".pdf"):
        loader = PyMuPDFLoader(os.path.join(pdf_dir, file))
        all_docs.extend(loader.load())


In [ ]:
# 3. Crear embeddings con HuggingFace

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(all_docs)

embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
)

# Crear y guardar vector store
db = FAISS.from_documents(chunks, embedding_model)
db.save_local("vectorstore")
print("✅ Vector store creado y guardado.")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.89k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ Vector store creado y guardado.


In [ ]:
# 4. Cargar LLM Groq (LLaMA 3)

chat = ChatGroq(
    temperature=0.3,
    groq_api_key=api_key,
    model_name="llama3-70b-8192"
)

In [ ]:
# 5. Crear Prompt especializado

template = """Sos un experto en educación en la primera infancia.
Responde en español de forma clara y basada únicamente en la información del contexto.

Pregunta: {question}

Contexto:
{context}
"""

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=template
)

In [ ]:
# 6. Construir la cadena RAG

db = FAISS.load_local("vectorstore", embedding_model, allow_dangerous_deserialization=True)

qa_chain = RetrievalQA.from_chain_type(
    llm=chat,
    retriever=db.as_retriever(search_kwargs={"k": 4}),
    chain_type_kwargs={"prompt": prompt},
    return_source_documents=True
)


In [ ]:
# 7. Consultas de prueba

from IPython.display import display, Markdown

# Pregunta 1
pregunta1 = "¿Qué cuidados básicos se deben tener en los primeros 1000 días de vida?"

# Ejecutar la consulta
resultado1 = qa_chain.invoke(pregunta1)
respuesta1 = resultado1['result']

# Mostrar la pregunta
display(Markdown("### **Pregunta 1:**"))
display(Markdown(f"{pregunta1}"))

# Mostrar la respuesta
display(Markdown("### **Respuesta generada por Groq (LLaMA 3)**"))
display(Markdown(f"```\n{respuesta1}\n```"))


### **Pregunta 1:**

¿Qué cuidados básicos se deben tener en los primeros 1000 días de vida?

### **Respuesta generada por Groq (LLaMA 3)**

```
Excelente pregunta! En los primeros 1000 días de vida, es fundamental brindar cuidados básicos que garanticen el desarrollo saludable del niño. A continuación, se presentan algunos de los cuidados básicos que se deben tener en cuenta:

1. **Seguridad emocional**: Los cuidadores deben sentirse seguros y con capacidad para tomar decisiones que beneficien al niño.
2. **Alimentación adecuada**: Permitir al niño comer solo, aunque derrame parte de la comida, y alentar a la familia a realizar actividades que fomenten hábitos saludables en la casa.
3. **Higiene personal**: Ayudar al niño a lavar y secar sus manos y cara, y sentarlo en la pelela por algunos minutos, varias veces al día, en horarios fijos.
4. **Apoyo emocional**: Brindar apoyo emocional al niño y a la familia, especialmente en situaciones de riesgo, como problemas perinatales.
5. **Acceso a servicios de salud**: Los servicios y profesionales de salud deben crear un entorno propicio antes del nacimiento, en el parto y en los primeros meses posnatales, brindando información y asesoramiento a los cuidadores.
6. **Intervenciones neonatales**: Realizar intervenciones durante el período neonatal, como el método canguro, que mejoran los resultados neonatales en los bebés pequeños y tienen efectos benéficos a largo plazo.

Es importante destacar que estos cuidados básicos deben ser brindados en un entorno de seguridad emocional, económica y social, que permita a los cuidadores participar en redes sociales y tomar decisiones que beneficien al niño.
```

In [ ]:
# Pregunta 2
pregunta2 = "¿Cómo estimular el lenguaje en un bebé de 1 año?"

# Ejecutar la consulta
resultado2 = qa_chain.invoke(pregunta2)
respuesta2 = resultado2['result']

# Mostrar la pregunta
display(Markdown("### **Pregunta 2:**"))
display(Markdown(f"{pregunta2}"))

# Mostrar la respuesta
display(Markdown("### **Respuesta generada por Groq (LLaMA 3)**"))
display(Markdown(f"```\n{respuesta2}\n```"))

### **Pregunta 2:**

¿Cómo estimular el lenguaje en un bebé de 1 año?

### **Respuesta generada por Groq (LLaMA 3)**

```
Excelente pregunta. Según el contexto, para estimular el lenguaje en un bebé de 1 año, se pueden implementar las siguientes estrategias:

1. **Responder positivamente**: Actuar con entusiasmo, sonriendo y hablando con el bebé cuando hace sonidos. Esto enseña al bebé a tomar turnos "hablando" de ida y vuelta en conversaciones.
3. **Repetir sonidos y palabras**: Repetir los sonidos del bebé y decir palabras simples que utilicen esos sonidos. Por ejemplo, si el bebé dice "bababa", repetir "bababa" y luego decir "libro".
4. **Hablar, leer y cantar**: Hablar, leer y cantar al bebé para ayudarle a desarrollar y entender el lenguaje.

Recuerda que también es importante responder a las necesidades emocionales del bebé, como pasar tiempo acurrucado y sosteniéndolo, lo que ayuda a sentirse seguro y cuidado.
```

In [ ]:
# Pregunta 3
pregunta3 = "¿Qué vacunas son obligatorias en los primeros dos años?"

# Ejecutar la consulta
resultado3 = qa_chain.invoke(pregunta3)
respuesta3 = resultado3['result']

# Mostrar la pregunta
display(Markdown("### **Pregunta 3:**"))
display(Markdown(f"{pregunta3}"))

# Mostrar la respuesta
display(Markdown("### **Respuesta generada por Groq (LLaMA 3)**"))
display(Markdown(f"```\n{respuesta3}\n```"))

### **Pregunta 3:**

¿Qué vacunas son obligatorias en los primeros dos años?

### **Respuesta generada por Groq (LLaMA 3)**

```
Según el contexto, las vacunas obligatorias en los primeros dos años son:

* IPV (Inactivada Polio Virus)
* HB (Hepatitis B)
* DPaT (Difteria, Pértussis y Tétanos)
* Neumo 23 valente (Neumococo)
* Varicela
* Hepatitis A

Es importante destacar que estas vacunas son obligatorias para la prevención de enfermedades graves y que la implementación de programas de vacunación ha permitido disminuir la mortalidad en menores de 5 años en Paraguay.
```

In [ ]:
# Pregunta 4
pregunta4 = "¿Cuál es la importancia de la lactancia?"

# Ejecutar la consulta
resultado4 = qa_chain.invoke(pregunta4)
respuesta4 = resultado4['result']

# Mostrar la pregunta
display(Markdown("### **Pregunta 4:**"))
display(Markdown(f"{pregunta4}"))

# Mostrar la respuesta
display(Markdown("### **Respuesta generada por Groq (LLaMA 3)**"))
display(Markdown(f"```\n{respuesta4}\n```"))

### **Pregunta 4:**

¿Cuál es la importancia de la lactancia?

### **Respuesta generada por Groq (LLaMA 3)**

```
La lactancia materna es fundamental para el desarrollo saludable del niño, ya que la leche materna es el alimento óptimo para casi todos los recién nacidos. La lactancia materna exclusiva y continua después de los 6 meses tiene múltiples beneficios, como la prevención de diarreas en niños mayores de 5 años, entre otros. Además, la lactancia materna requiere un cuidado cariñoso y sensible en casa, y es fundamental que las madres reciban apoyo idóneo para suministrar leche materna a sus hijos, especialmente en el caso de bebés prematuros y de bajo peso al nacer.
```

In [ ]:
# Pregunta 5
pregunta5 = "¿Cuáles son las etapas por la que pasa un bebé antes de caminar?"

# Ejecutar la consulta
resultado5 = qa_chain.invoke(pregunta5)
respuesta5 = resultado5['result']

# Mostrar la pregunta
display(Markdown("### **Pregunta 5:**"))
display(Markdown(f"{pregunta5}"))

# Mostrar la respuesta
display(Markdown("### **Respuesta generada por Groq (LLaMA 3)**"))
display(Markdown(f"```\n{respuesta5}\n```"))

### **Pregunta 5:**

¿Cuáles son las etapas por la que pasa un bebé antes de caminar?

### **Respuesta generada por Groq (LLaMA 3)**

```
Excelente pregunta! Antes de caminar, un bebé pasa por varias etapas importantes para desarrollar sus habilidades motoras y de equilibrio. A continuación, te presento las etapas que un bebé pasa antes de caminar:

1. **Boca arriba**: En esta posición, el bebé lleva los pies a la boca con un ligero estímulo, lo que ayuda a fortalecer los músculos de las piernas y a desarrollar la coordinación motora.
2. **Sentado**: En esta etapa, el bebé se ladea hacia un lado y extiende los brazos hacia la posición adecuada para protegerse de la caída, lo que desarrolla su equilibrio y coordinación motora.
4. **De pie, sujetándolo del tronco**: En esta posición, el bebé realiza movimientos de flexión y extensión de piernas, lo que ayuda a fortalecer los músculos de las piernas y a desarrollar su equilibrio.
5. **Se sienta estable** (alrededor de 7-8 meses): En esta etapa, el bebé se siente estable y seguro en su posición sentada, lo que es un paso importante hacia la marcha.
6. **Gatea o se arrastra** (alrededor de 9 meses): En esta etapa, el bebé comienza a moverse hacia adelante, lo que desarrolla su coordinación motora y equilibrio.
7. **Camina para atrás): En esta etapa, el bebé da dos pasos para atrás sin caerse, lo que indica que ha desarrollado la habilidad de caminar.

Es importante destacar que cada bebé desarrollarse a su propio ritmo, y es fundamental la estimulación oportuna y adecuada para apoyar su desarrollo.
```

In [ ]:
#8. Evaluación del modelo

!pip install -U bert_score rouge-score


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 4.8 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=48d6d10fa579acf7afb0f486d28f06d752cf6623a8521a93c309cf82408d1bd2
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score


In [ ]:
# Listado de preguntas
preguntas = [
    "¿Qué cuidados básicos se deben tener en los primeros 1000 días de vida?",
    "¿Cómo estimular el lenguaje en un bebé de 1 año?",
    "¿Qué vacunas son obligatorias en los primeros dos años?",
    "¿Cuál es la importancia de la lactancia?",
    "¿Cuáles son las etapas por la que pasa un bebé antes de caminar?"
]

# Listado de respuestas
respuestas = []

# Lista para guardar contexto
contextos = []

# Repetir cada consulta y extraer información
for pregunta in preguntas:
    resultado = qa_chain.invoke(pregunta)

    respuestas.append(resultado["result"])
    contextos.append([doc.page_content for doc in resultado["source_documents"]])  # lista de textos recuperados

# Verificamos el primer resultado
print("📘 Ejemplo de respuesta:")
print(respuestas[0])
print("\n📚 Ejemplo de contexto recuperado:")
print(contextos[0][0][:500] + "...")


📘 Ejemplo de respuesta:
Excelente pregunta! En los primeros 1000 días de vida, es fundamental brindar cuidados básicos que apoyen el desarrollo saludable del niño. A continuación, se presentan algunos de los cuidados básicos que se deben tener en cuenta:

1. **Seguridad emocional**: Los cuidadores deben sentirse seguros y capaces de brindar un ambiente emocional estable para el niño.
2. **Alimentación adecuada**: Permitir al niño comer solo, aunque derrame parte de la comida, y alentar a la familia a realizar actividades que fomenten hábitos saludables en la casa.
3. **Higiene personal**: Ayudar al niño a lavar y secar sus manos y su cara, y sentarlo en la pelela por algunos minutos, varias veces al día, en horarios fijos.
4. **Apoyo emocional**: Brindar apoyo emocional a los cuidadores, especialmente en situaciones de necesidad, como en el caso de bebés con problemas perinatales.
5. **Acceso a servicios de salud**: Asegurarse de que los cuidadores tengan acceso a servicios de salud y 

In [ ]:
from bert_score import score as bert_score
from rouge_score import rouge_scorer
import pandas as pd

# Respuestas ideales o referencias humanas
respuestas_referencia = [
    "Durante los primeros 1000 días de vida, es esencial brindar cuidados básicos que aseguren un desarrollo saludable en el niño. Entre ellos se destacan una alimentación adecuada que fomente la autonomía, hábitos de higiene como el lavado de manos y cara, y la estimulación sensorial mediante rutinas como sentarlo en la pelela en horarios fijos. También es clave ofrecer acompañamiento emocional que le brinde seguridad y afecto, así como asesorar a las familias, especialmente en casos de dificultades perinatales. Además, los profesionales de salud deben generar un entorno favorable desde el embarazo hasta los primeros meses de vida. Todo esto debe darse en un contexto de amor, respeto y apoyo constante, ya que un entorno seguro es fundamental para el bienestar físico, emocional y cognitivo del niño.",
    "Para estimular el lenguaje en un bebé de un año, es fundamental interactuar con él de forma positiva y constante. Responder con entusiasmo a sus sonidos, sonreírle y hablarle ayuda a que aprenda a turnarse en una conversación. También es útil repetir los sonidos que emite y añadir palabras simples que los contengan, como decir “libro” después de que diga “bababa”. Además, hablarle frecuentemente, leerle cuentos y cantarle canciones son actividades que favorecen la comprensión y el desarrollo del lenguaje. La clave está en mantener una interacción cercana y afectuosa, ya que esta etapa es crucial para su aprendizaje comunicativo.",
    "Las vacunas obligatorias en los primeros dos años incluyen Hepatitis B, Polio, DPaT, Neumococo, Varicela y Hepatitis A según el esquema nacional.",
    "La lactancia materna desempeña un papel crucial durante la primera infancia, ya que ofrece el alimento más adecuado para los recién nacidos. Es especialmente beneficiosa para bebés prematuros o con bajo peso al nacer. Mantener la lactancia materna de forma exclusiva desde el nacimiento y prolongarla más allá de los 6 meses aporta numerosos beneficios para la salud y el desarrollo del niño. Además, constituye una medida importante para prevenir la diarrea en niños mayores de cinco años. Por todo esto, fomentar y respaldar la lactancia materna es fundamental para asegurar el bienestar infantil.",
    "Antes de comenzar a caminar, un bebé atraviesa una serie de etapas fundamentales en su desarrollo motor. Inicialmente, en posición boca arriba, responde a estímulos llevando los pies a la boca. Luego, empieza a sentarse ladeándose y extendiendo los brazos para mantener el equilibrio. Posteriormente, con apoyo en el tronco, se pone de pie y realiza movimientos de flexión y extensión de las piernas. Alrededor de los 7 u 8 meses, logra sentarse de forma estable, y hacia los 9 meses comienza a gatear o arrastrarse para desplazarse por sí solo. Finalmente, aprender a caminar hacia atrás marca un hito importante en el desarrollo de su coordinación y control corporal."
]

# 1. Calcular BERTScore F1
_, _, f1_bert = bert_score(respuestas, respuestas_referencia, lang="es")

# 2. Calcular ROUGE-L F1
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
rouge_scores = [scorer.score(ref, pred)['rougeL'].fmeasure for ref, pred in zip(respuestas_referencia, respuestas)]

# 3. Consolidar en DataFrame
df_eval = pd.DataFrame({
    "Pregunta": preguntas,
    "BERTScore F1": [round(f.item(), 3) for f in f1_bert],
    "ROUGE-L F1": [round(r, 3) for r in rouge_scores]
})

# Mostrar tabla
df_eval


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

,Pregunta,BERTScore F1,ROUGE-L F1
0,¿Qué cuidados básicos se deben tener en los pr...,0.746,0.309
1,¿Cómo estimular el lenguaje en un bebé de 1 año?,0.721,0.316
2,¿Qué vacunas son obligatorias en los primeros ...,0.802,0.419
3,¿Cuál es la importancia de la lactancia?,0.774,0.329
4,¿Cuáles son las etapas por la que pasa un bebé...,0.720,0.271
